# Device

In [ ]:
import socket
import utils

k_mb = utils.generate_key()

# Simplified reception of data; in practice, set up a connection
data = received_data()
nonce, m1_h1 = data[:16], data[16:]
m1, h1 = m1_h1[:-32], m1_h1[-32:]

decrypted_data = utils.decrypt(m1, k_mb, nonce)
if utils.hash1(nonce, decrypted_data, k_mb) == h1:
    print("Request Verified:", decrypted_data.decode())
